In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [1]:
def get_route_distance(api_key, start_coordinate, end_coordinate):

    # Define the API endpoint for route calculation

    endpoint = 'https://api.openrouteservice.org/v2/directions/driving-car'

    # Define the coordinates for the starting and ending points of the route
    start_coords = start_coordinate # [longitude, latitude]
    end_coords = end_coordinate    # [longitude, latitude]

    payload = {
        'coordinates': [start_coords, end_coords],
        'profile': 'driving-car',  # You can change this to 'foot-walking' or 'cycling-regular' for different modes of transport
    }

    # Set the API key in the headers
    headers = {
        'Authorization': f'Bearer {api_key}',
    }

    # Send a POST request to the OpenRouteService API
    response = requests.post(endpoint, json=payload, headers=headers)

    # Check if the request was successful (HTTP status code 200)
    if response.status_code == 200:
        data = response.json()

        if len(data['routes'][0]['summary']) == 0:
            distance = 0

        else:

            distance = data['routes'][0]['summary']['distance']

        return int(distance//1000)

    else:
        return f"Error: {response.status_code} - {response.text}"

In [2]:
import csv
from geopy.distance import geodesic
import requests
import json
import time

# Dictionary containing city names as keys and their coordinates as values
cities = {'Lucknow': (26.850000, 80.949997), 'Rajsamand': (25.066700, 73.883301), ...}

# Loop through each city in the dictionary
for city_name, city_coordinates in cities.items():

    # Calculate the time interval for API requests (70 seconds divided by 40 requests)
    time_interval = 70 / 40

    # API keys for route distance calculations
    api_key = '5b3ce3597851110001cf624887280eac23374df08b057352fff2310a'
    api_key2 = '5b3ce3597851110001cf624896d87d32d9f040c58bc9ec95b252fa7d'

    # List to store CSV data
    csv_data = []

    # Loop through all cities to calculate distances
    for city, coordinates in cities.items():

        try:
            # Get the start and end coordinates for distance calculation
            start_coordinate = [city_coordinates[1], city_coordinates[0]]
            end_coordinate = [coordinates[1], coordinates[0]]

            # Measure the start time for API request timing
            start_time = time.time()

            # Get the route distance using the first API key
            route_distance = get_route_distance(api_key, start_coordinate, end_coordinate)

            # Measure the end time for API request timing
            end_time = time.time()

            # Calculate the geodesic distance between the two cities
            distance_km = geodesic(city_coordinates, coordinates).km

            # Calculate the time taken for the API request
            request_time = end_time - start_time

            # If the request time is less than the time interval, sleep to avoid rate limiting
            if request_time < time_interval:
                time.sleep(time_interval - request_time)
            
            # Append data to the CSV data list
            csv_data.append([city_name, city, int(distance_km), int(route_distance)])

        except:
            # If an exception occurs, use the second API key for route distance
            start_coordinate = [city_coordinates[1], city_coordinates[0]]
            end_coordinate = [coordinates[1], coordinates[0]]
            start_time = time.time()
            route_distance = get_route_distance(api_key2, start_coordinate, end_coordinate)
            end_time = time.time()
            distance_km = geodesic(city_coordinates, coordinates).km
            request_time = end_time - start_time

            # If the request time is less than the time interval, sleep to avoid rate limiting
            if request_time < time_interval:
                time.sleep(time_interval - request_time)

            # Append data to the CSV data list
            csv_data.append([city_name, city, int(distance_km), int(route_distance)])

    # Define the CSV file name based on the city
    csv_file_name = f"{city_name}.csv"

    # Open the CSV file for writing
    with open(csv_file_name, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # Write the header row
        csv_writer.writerow(["node1", "node2", "Heuristic distance", "route_distance"])

        # Write the data rows
        csv_writer.writerows(csv_data)

    # Print a message indicating that the CSV file has been created
    print(f"CSV file '{csv_file_name}' has been created.")


CSV file 'Lucknow.csv' has been created.
